# CV Final Project: Reading Time By Aviv Bar-on#

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from pynput.mouse import Button, Controller

1. Load a Classifiers for the face and eyes detection. Using 'haarcascades'.

In [ ]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")

2. Initialize mouse controller. Using pynput library.

In [ ]:
mouse = Controller()

3. Initialize detector parameters.

In [ ]:
detector_params = cv2.SimpleBlobDetector_Params()
detector_params.filterByArea = True
detector_params.maxArea = 1500
detector = cv2.SimpleBlobDetector_create(detector_params)

Detect Faces Function: Using the cascade classifier for face detecting.
The function get an image and classifier and return the frame of the biggest detected face because there some things in the background that seems to look like face sometime.

In [ ]:
def detect_faces(img, classifier):
    gray_frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    coords = face_cascade.detectMultiScale(gray_frame, 1.3, 5)
    if len(coords) > 1:
        biggest = (0, 0, 0, 0)
        for i in coords:
            if i[3] > biggest[3]:
                biggest = i
        biggest = np.array([i], np.int32)
    elif len(coords) == 1:
        biggest = coords
    else:
        return None
    for (x, y, w, h) in biggest:
        frame = img[y:y + h, x:x + w]
    return frame

Detect Eyes Funtion: Using cascade classifier to the detect the eyes area and then find both eyes.
The function get an image and classifier and return seperate the left eye and right eye.

In [ ]:
def detect_eyes(img, classifier):
    gray_frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray_frame, 1.3, 7) # detect eyes
    width = np.size(img, 1) # get face frame width
    height = np.size(img, 0) # get face frame height
    left_eye = None
    right_eye = None
    for (x, y, w, h) in eyes:
        if y > height / 2:
            pass
        eyecenter = x + w / 2  # get the eye vertical center, to seperate the eyes
        if eyecenter < width * 0.5:
            left_eye = img[y:y + h, x:x + w]
        else:
            right_eye = img[y:y + h, x:x + w]
    return left_eye, right_eye

Cut eyebrows function: the eyebrows always take ~25% of the image starting from the top, so if we will take out this part it will be more easy to detect the pupil.

In [ ]:
def cut_eyebrows(img):
    height, width = img.shape[:2]
    eyebrow_h = int(height / 4)
    img = img[eyebrow_h:height, 0:width]  # cut eyebrows out (15 px)
    return img

Blob Process function: Processing the image of the eyes to detect the pupil.
The function return keypoints of the pupils.

In [ ]:
def blob_process(img, threshold, detector):
    gray_frame = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, img = cv2.threshold(gray_frame, threshold, 255, cv2.THRESH_BINARY)
    kernel = np.ones((3, 3), np.uint8)
    img = cv2.bilateralFilter(img, 10, 15, 15)
    img = cv2.erode(img, kernel, iterations=2)
    img = cv2.dilate(img, kernel, iterations=4)
    img = cv2.medianBlur(img, 5)
    keypoints = detector.detect(img)
    return keypoints

The issue with OpenCV track bars is that they require a function that will happen on each track bar movement. We don’t need any sort of action, we only need the value of our track bar, so we create a nothing() function:

In [ ]:
def nothing(x):
    pass

Main Function: We are using laptop's webcam to detect our eyes while reading something in the laptop so that when we are looking down and got to the bottom of the screen, the cam will detect that and scroll the screen up for us.

Threshold Bar: The threshold need to change because of the light or distance from the laptop, there is a bar that make it easy to siut the threshold for the specific condition.

While we are reading the frames from the camera, the frames go through face detecting, eye detecting in face, deleting eyebrows and finally processed to find the keypoints of the pupils.

Later, we are using one of our keypoints parameter that change when we are looking down the screen and gets bigger. when that parameter is over 15.5 the screen will scroll up.


In [ ]:
cap = cv2.VideoCapture(0)
cv2.namedWindow('Eye Detector')
cv2.createTrackbar('threshold', 'Eye Detector', 42, 255, nothing) # the threshold is changing by light and distance
y = 0 # Importent when the first frame will not detect our eyes

while True:
    ret,frame = cap.read()

    if(ret == True): 
        face_frame = detect_faces(frame, face_cascade)
        if face_frame is not None:
            eyes = detect_eyes(face_frame, eye_cascade)
            
            for eye in eyes:
                if eye is not None:
                    threshold = cv2.getTrackbarPos('threshold', 'Eye Detector')
                    eye = cut_eyebrows(eye)
                    keypoints = blob_process(eye, threshold, detector)
                    eye = cv2.drawKeypoints(eye, keypoints, eye, (0, 0, 255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
            
            for keypoint in keypoints:
                y = keypoint.pt[1]
                
            if y is None:
                print("None")
                
            elif y > 15.5:
                mouse.scroll(0,-0.3)
                
        cv2.imshow('Eye Detector', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break



cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1);